# Thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import nltk
import numpy as np
import pandas as pd
from collections import Counter
import bisect
import re
import os
import pickle
from nltk.corpus import stopwords
import math
from nltk.stem import WordNetLemmatizer
import gensim
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

ss = SnowballStemmer("english")

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

# Tiền xử lý

In [ ]:
def preprocess_text(text):
    processed_text = text.lower()
    processed_text = processed_text.replace("’", "'")
    processed_text = processed_text.replace("“", '"')
    processed_text = processed_text.replace("”", '"')
    non_words = re.compile(r"[^A-Za-z']+")
    processed_text = re.sub(non_words, ' ', processed_text)
    return processed_text

In [ ]:
def get_words_from_text(text):
    processed_text = preprocess_text(text)
    filtered_words = [word for word in processed_text.split() if word not in stopwords.words('english')]
    processed_words = []
    for i in range(len(filtered_words)):
        #processed_words.append(lemmatizer.lemmatize(filtered_words[i]))
        #processed_words.append(ps.stem(filtered_words[i]))
        processed_words.append(ss.stem(filtered_words[i]))
    return processed_words

In [ ]:
def get_text_from_file(filename):
    with open(filename, encoding='utf-8', mode='r') as f:
        text = f.read()
    return text

In [ ]:
def indexing(docs_path):
    terms = []
    index = []
    norm_list = []
    N = 0

    # Indexing postings
    for doc_file in sorted(os.listdir(docs_path), key=lambda x: int(os.path.splitext(x)[0])):
        filename = os.path.join(docs_path, doc_file)
        N += 1
        text = get_text_from_file(filename)
        words = get_words_from_text(text)

        for word in words:
            if word not in terms:
                terms.append(word)
                index.append([(N, 1)])

            else:
                temp_index = terms.index(word)
                if N not in [i[0] for i in index[temp_index]]:
                    index[temp_index].append((N, 1))
                    continue

                for i in range(len(index[temp_index])):
                    if index[temp_index][i][0] == N:
                        index[temp_index][i] = (N, index[temp_index][i][1] + 1)
                        break

    # Calculate weights
    for i in range(len(index)):
        for j in range(len(index[i])):
            temp = list(index[i][j])
            temp[1] = index[i][j][1] * math.log(N / len(index[i]) + 1) # Formula: TF * IDF (IDF = log(N / ndoc(t)))
            index[i][j] = tuple(temp)

    # Normalization
    for i in range(N):
        norm = 0

        for j in range(len(index)):
            temp = [item for item in index[j] if item[0] == i+1]

            if len(temp) != 0:
                norm += math.pow(temp[0][1], 2)

        norm_list.append(math.sqrt(norm))

    for i in range(len(index)):
        for j in range(len(index[i])):
            temp = list(index[i][j])
            temp[1] = index[i][j][1] / norm_list[index[i][j][0]-1]
            index[i][j] = tuple(temp)

    return terms, index

In [ ]:
terms, index = indexing(r"/content/drive/MyDrive/CS419/Dataset/Cranfield/docs")

In [ ]:
print(len(terms))

4257


In [ ]:
print(index)

[[(1, 0.081266887536544), (11, 0.05517871668228564), (12, 0.04617529168402256), (16, 0.0502155118910492), (17, 0.048833229383609546), (19, 0.09544726095541167), (25, 0.027211912270156558), (29, 0.032128927975605764), (30, 0.10397614891240953), (35, 0.05100208647355126), (37, 0.04809593464968989), (41, 0.07545716015467929), (42, 0.01842325586203017), (43, 0.04441395661085414), (47, 0.03381180974342808), (52, 0.044956427947928305), (53, 0.03233982627926913), (58, 0.04251864888070988), (69, 0.04831317611090757), (70, 0.02978418842997248), (74, 0.07504130389783262), (78, 0.06775393501523116), (84, 0.09260938385985695), (99, 0.05374061273613475), (101, 0.027539201458194565), (103, 0.038874347770346644), (112, 0.042093368733166076), (115, 0.02951893806755267), (121, 0.053454898900573526), (123, 0.10587344014571334), (131, 0.027581367675031374), (137, 0.10454557183689633), (140, 0.022223892887014412), (142, 0.0622306286517538), (154, 0.06591279130883423), (156, 0.02051463546417389), (167, 0.0

# Truy vấn

In [ ]:
def load_query(path):
    query = []

    for doc_file in sorted(os.listdir(path),key=lambda x: int(os.path.splitext(x)[0])):
        filename = os.path.join(path, doc_file)

        with open(filename, encoding='utf-8', mode='r') as f:
            temp2 = f.readlines()

        listToStr = ' '.join([str(elem) for elem in temp2])
        temp = ""

        for i in listToStr.split():
          content = i.replace('.', '').replace("''", '').replace('\n', '').lower()
          content = gensim.utils.simple_preprocess(content)
          content = [' '+non_stopword for non_stopword in content if non_stopword not in stop_words]
          #content = [ps.stem(word) for word in content]
          #content = [lemmatizer.lemmatize(word) for word in content]
          content = [ss.stem(word) for word in content]
          temp +=  " ".join(content)

        query.append(temp)

    return query

In [ ]:
def load_res(res_path):
  goal = []

  for doc_file in sorted(os.listdir(res_path),key=lambda x: int(os.path.splitext(x)[0])):
    filename = os.path.join(res_path, doc_file)

    with open(filename, encoding='utf-8', mode='r') as f:
      temp2 = f.readlines()

    temp3 = []
    a = []

    for x in temp2:
      a = x.strip().split()

      for i in a:
        temp3.append(int(i))

    goal.append(temp3)

  return goal

In [ ]:
def get_search_results(query, terms, index):
    words = get_words_from_text(query)

    # Get postings
    temp_postings = {}
    temp_words = list(set(words))

    for i in range(len(temp_words)):
        if temp_words[i] in terms:
            temp_postings[temp_words[i]] = index[terms.index(temp_words[i])].copy()

    # Calculating TF * IDF (IDF = log(N / ndoc(t)))
    N = []
    for i in range(len(index)):
        N.append(max(index[i],key=lambda item:item[0])[0])

    N = max(N)
    temp = [item for item in words if item in terms]

    weight_list = Counter(temp)
    weight_list = {x: weight_list[x] * math.log(N / len(temp_postings[x])) for x in weight_list}

    # Normalization
    norm = 0
    temp = list(weight_list.values())

    for i in range(len(temp)):
        norm += math.pow(temp[i], 2)

    norm = math.sqrt(norm)
    weight_list = {x: weight_list[x] / norm for x in weight_list}

    for i in temp_postings:
        for j in range(len(temp_postings[i])):
            temp = list(temp_postings[i][j])
            temp[1] *= weight_list[i]
            temp_postings[i][j] = tuple(temp)

    # Get search result(s)
    res = list(temp_postings.values())
    result = {}

    for L in res:
        for key, value in L:
            result[key] = result.get(key, 0) + value

    result = list(result.items())


    #print('Result contains ', len(result))


    return sorted(result, key=lambda tup: tup[1], reverse=True)

In [ ]:
def evaluateMAP(test_query, test_path, terms, index):
    PRECISION = []
    RECALL = []
    query = load_query(test_query)

    goal = load_res(test_path)

    average_precision = []

    for i in range(len(goal)):
        temp4 = get_search_results(query[i], terms, index)
        temp5 = [i[0] for i in temp4] # lấy ra danh sách các document trả về sắp xếp theo độ phù hợp giảm dần

        precision = []
        recall = []
        count = 0
        n = 0
        temp_precision = []

        # tính AP và recall
        for j in range(len(temp5)):
            if temp5[j] in goal[i]:
                count += 1

            n += 1
            recall.append(count / len(goal[i]))
            precision.append(count / n)

        # tính AP nội suy
        temp = 0
        for i in range(len(precision)-1, -1, -1):
            if temp >= precision[i]:
                precision[i] = temp

            elif temp < precision[i]:
                temp = precision[i]

        # tính MAP 11 điểm
        precision_list_11 = []
        points = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

        for idx, i in enumerate(points):
            for j in range(len(recall)):
                if recall[j] >= i:
                    precision_list_11.append(max(precision[j:]))
                    break

        if len(precision_list_11) < 11:
          precision_list_11 += [0]*(11-len(precision_list_11))


        if len(precision_list_11) != 0:
            average_precision.append(sum(precision_list_11) / 11)

        else:
            average_precision.append(0)

        PRECISION.append(precision[-1])
        RECALL.append(recall[-1])

    if len(average_precision) != 0:
        return sum(average_precision) / len(average_precision), sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)

    else:
        return 0, sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)

In [ ]:
queries_path = '/content/drive/MyDrive/CS419/Dataset/Cranfield/queries'
qrel_path = '/content/drive/MyDrive/CS419/Dataset/Cranfield/qrel'

In [ ]:
import time
start_time = time.time()
mAP, precision, recall = evaluateMAP(queries_path, qrel_path , terms, index)

print('mAP = ', mAP )
print('Precision = ', precision )
print('Recall = ', recall )
print("Time: %s seconds" % (time.time() - start_time))

mAP =  0.40608942904531525
Precision =  0.00977702690433962
Recall =  0.9619441630740482
Time: 7.437735557556152 seconds


# Thử nghiệm


In [ ]:
docs_path = "/content/drive/MyDrive/CS419/Dataset/Cranfield/docs"
doc_file = "6.txt"
filename = os.path.join(docs_path, doc_file)

#text = get_text_from_file(filename)
text = "conventional aerofoils, (b) the low-speed flow about very thick aerofoils, e.g.,"
print(text)

conventional aerofoils, (b) the low-speed flow about very thick aerofoils, e.g.,


In [ ]:
import glob

ap = []
interpolated_ap = []
interpolated_ap_11 = []
count = 0

for path in glob.glob( "./fld_whoosh/*.txt"):
  with open(path, 'r') as f:
    returned_docs = list(map(int, f.read().split()))
    precision = []
    recall = []
    count = 0

    for i, doc in enumerate(returned_docs):
      doc_id =int(path[13:-4])
      if gt_matrix[doc_id - 1][doc-1]:
        count += 1
        recall.append(count / gt_dict[doc_id])
        precision.append(count / (i + 1))

    if len(precision) == 0:
      ap.append(0)
      interpolated_ap.append(0)
    else:
      ap.append(np.average(precision))
      interpolated_ap.append(np.average(calc_interpolated_map(precision)))
      interpolated_ap_11.append(np.average(calc_interpolated_map_11(recall, precision)))

print(f"MAP: {np.mean(ap)}")
print(f"Interpolated MAP: {np.mean(interpolated_ap)}")
print(f"Interpolated MAP 11 points: {np.mean(interpolated_ap_11)}")

MAP: nan
Interpolated MAP: nan
Interpolated MAP 11 points: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
words = get_words_from_text(text)
for word in words:
  #stop_words:
    print(word)

convent
aerofoil
b
low
speed
flow
thick
aerofoil
e
g


In [ ]:
for word in terms:
  print(word)

experiment
investig
aerodynam
wing
slipstream
studi
propel
made
order
determin
spanwis
distribut
lift
increas
due
differ
angl
attack
free
stream
veloc
ratio
result
intend
part
evalu
basi
theoret
treatment
problem
compar
span
load
curv
togeth
support
evid
show
substanti
increment
produc
destal
boundari
layer
control
effect
integr
remain
subtract
found
agre
well
potenti
flow
theori
empir
specif
configur
experi
simpl
shear
past
flat
plate
incompress
fluid
small
viscos
high
speed
viscous
two
dimension
bodi
usual
necessari
consid
shock
wave
emit
nose
lead
edg
consequ
exist
inviscid
rotat
region
situat
aris
instanc
hyperson
somewhat
prandtl
classic
origin
outsid
irrot
must
possibl
vortic
recent
discuss
ferri
libbi
present
paper
shown
treat
approxim
novel
featur
constant
restrict
steadi
equat
pressur
gradient
solut
laminar
thick
skin
friction
obtain
karman
pohlhausen
techniqu
comparison
uniform
also
one
transient
heat
conduct
doubl
slab
subject
linear
input
time
intern
analyt
composit
expos
s